In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Corrected dataset path (no space)
dataset_path = '/content/drive/MyDrive/DS_PROJECT/DS_PROJECT/SolarGuard/dataset'

In [3]:
import zipfile
import os

zip_paths = [
    f'{dataset_path}/Bird-drop-20250428T115508Z-1-001.zip',
    f'{dataset_path}/Clean-20250428T115502Z-1-001.zip',
    f'{dataset_path}/Dusty-20250428T115457Z-1-001.zip',
    f'{dataset_path}/Electrical-damage-20250428T115452Z-1-001.zip',
    f'{dataset_path}/Physical-Damage-20250428T115448Z-1-001.zip',
    f'{dataset_path}/Snow-Covered-20250428T115440Z-1-001.zip',
]

for zip_file in zip_paths:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(dataset_path)
        image_files = [f for f in zip_ref.namelist() if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"{zip_file.split('/')[-1]} → {len(image_files)} image(s)")

Bird-drop-20250428T115508Z-1-001.zip → 207 image(s)
Clean-20250428T115502Z-1-001.zip → 193 image(s)
Dusty-20250428T115457Z-1-001.zip → 190 image(s)
Electrical-damage-20250428T115452Z-1-001.zip → 103 image(s)
Physical-Damage-20250428T115448Z-1-001.zip → 69 image(s)
Snow-Covered-20250428T115440Z-1-001.zip → 123 image(s)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 224, 224  # Recommended size for MobileNet
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_generator.num_classes


Found 711 images belonging to 6 classes.
Found 174 images belonging to 6 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,734 (42.61 MB)

 Trainable params: 11,169,734 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
cnn_history = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 106s 4s/step - accuracy: 0.8777 - loss: 0.3666 - val_accuracy: 0.6494 - val_loss: 1.2623
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.9321 - loss: 0.2218 - val_accuracy: 0.6322 - val_loss: 1.2077
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 101s 4s/step - accuracy: 0.9449 - loss: 0.1733 - val_accuracy: 0.6034 - val_loss: 1.2770
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 101s 4s/step - accuracy: 0.9358 - loss: 0.1767 - val_accuracy: 0.6552 - val_loss: 1.4105
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 99s 4s/step - accuracy: 0.9433 - loss: 0.1634 - val_accuracy: 0.6264 - val_loss: 1.4358
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 101s 4s/step - accuracy: 0.9727 - loss: 0.1185 - val_accuracy: 0.6494 - val_loss: 1.3627
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 101s 4s/step - accuracy: 0.9465 - loss: 0.1538 - val_accuracy: 0.6207 - val_loss: 1.4646
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 100s 4s/step - accuracy: 0.9689 - loss: 0.1115 - val_accuracy: 0.6437 - va

In [8]:
# Evaluate CNN model on validation data
cnn_loss, cnn_accuracy = cnn_model.evaluate(val_generator)
print(f"CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")


6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6352 - loss: 1.8692
CNN Model Accuracy: 63.22%


In [20]:
cnn_model.save('Sequential_model.h5')

In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze for feature extraction

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=predictions)

mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mobilenet_model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_mobilenet.h5', save_best_only=True, monitor='val_loss')


In [13]:

mobilenet_history = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9901 - loss: 0.0682

23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9900 - loss: 0.0682 - val_accuracy: 0.8333 - val_loss: 0.6666
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9933 - loss: 0.0420

23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9928 - loss: 0.0428 - val_accuracy: 0.8391 - val_loss: 0.6036
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9923 - loss: 0.0600

23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9920 - loss: 0.0602 - val_accuracy: 0.8563 - val_loss: 0.5527
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9873 - loss: 0.0603 - val_accuracy: 0.8736 - val_loss: 0.5699
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9952 - loss: 0.0424 - val_accuracy: 0.8621 - val_loss: 0.6041
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.9903 - loss: 0.0401 - val_accuracy: 0.8506 - val_loss: 0.6111
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.9929 - loss: 0.0354 - val_accuracy: 0.8506 - val_loss: 0.6173
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.9962 - loss: 0.0393 - val_accuracy: 0.8506 - val_loss: 0.6426


In [14]:

# Fine-tuning MobileNetV2
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - accuracy: 0.6107 - loss: 1.5916 - val_accuracy: 0.4310 - val_loss: 13.0170
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.8190 - loss: 0.5458 - val_accuracy: 0.4425 - val_loss: 14.9911
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.9184 - loss: 0.3051 - val_accuracy: 0.3908 - val_loss: 15.6069
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9158 - loss: 0.3197 - val_accuracy: 0.5057 - val_loss: 8.5347
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.9577 - loss: 0.1582 - val_accuracy: 0.4598 - val_loss: 9.6807
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.9548 - loss: 0.1805 - val_accuracy: 0.4368 - val_loss: 13.8498
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.9005 - loss: 0.3502 - val_accuracy: 0.3563 - val_loss: 21.0475
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.9355 - loss: 0.2271 - val_accuracy: 0.4655 - val_

In [15]:
# Evaluate MobileNetV2 model on validation data
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(val_generator)
print(f"MobileNetV2 Model Accuracy: {mobilenet_accuracy * 100:.2f}%")


6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5022 - loss: 8.5829
MobileNetV2 Model Accuracy: 50.57%


In [10]:
mobilenet_history = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.2866 - loss: 1.9150 - val_accuracy: 0.6552 - val_loss: 0.9684
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.6578 - loss: 0.9803 - val_accuracy: 0.7356 - val_loss: 0.7107
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.7268 - loss: 0.7735 - val_accuracy: 0.7644 - val_loss: 0.6699
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.7797 - loss: 0.6031 - val_accuracy: 0.7816 - val_loss: 0.6017
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.7759 - loss: 0.6051 - val_accuracy: 0.8103 - val_loss: 0.5766
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.8350 - loss: 0.5088 - val_accuracy: 0.8161 - val_loss: 0.5363
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.8425 - loss: 0.4592 - val_accuracy: 0.8046 - val_loss: 0.5600
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9061 - loss: 0.3386 - val_accuracy: 0.8103 - val_loss:

In [11]:
# Evaluate MobileNetV2 model on validation data
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(val_generator)
print(f"MobileNetV2 Model Accuracy: {mobilenet_accuracy * 100:.2f}%")


6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8470 - loss: 0.5645
MobileNetV2 Model Accuracy: 84.48%


In [19]:
mobilenet_model.save('mobile_net_model.h5')